# Experiment with handling nans in healpix data

* A single nan in high-res data causes issues with global means when you coarsen the data
* E.g. np.nanmean(field.coarsen()) != np.nanmean(field)
* To handle this, you need to store the weights corresponding to the number of nans that have been averaged over to coarsen the data
* And to calc global means, use these weights to perform weighted mean
* This notebook demonstrates how to do this using numpy/xarray

In [1]:
import numpy as np
import xarray as xr

In [2]:
def arr_all_close(a):
    return np.isclose(a[:-1], a[1:]).all()

print(arr_all_close([1, 1, 1+ 1e-7]))
print(arr_all_close([1, 1, 1+ 1e-2]))

True
False


In [3]:
# Basic idea: Set up a field at zoom = 2, assign a nan, and check global mean at different zoom levels.
az2 = np.arange(12 * 4**2).astype(float)
az2[6] = np.nan

# Simple coarsen of 1D data by reshaping and taking mean along dummy axis.
az1 = np.nanmean(az2.reshape(-1, 4), axis=-1)
# Calc the fraction of nans for each coarsened grid cell. 1 nan -> 3/4.
wz1 = 1 - np.isnan(az2.reshape(-1, 4)).sum(axis=-1) / 4

# Apply weights to do coarsening. 
az0 = (az1 * wz1).reshape(-1, 4).sum(axis=-1) / wz1.reshape(-1, 4).sum(axis=-1)
# Coarsen weights. No nans in these.
wz0 = wz1.reshape(-1, 4).mean(axis=-1)

# Check output.
print(np.nanmean(az2))
print(np.average(az1, weights=wz1))
print(np.average(az0, weights=wz0))
print('all_close:', arr_all_close([np.nanmean(az2), np.average(az1, weights=wz1), np.average(az0, weights=wz0)]))

95.96858638743456
95.96858638743456
95.96858638743456
all_close: True


In [4]:
# Numpy func.
def hp_coarsen_with_weights(field, weights=None):
    if weights is None:
        coarse_field = np.nanmean(field.reshape(-1, 4), axis=-1)
        new_weights = 1 - np.isnan(field.reshape(-1, 4)).sum(axis=-1) / 4
    else:
        coarse_field = (field * weights).reshape(-1, 4).sum(axis=-1) / weights.reshape(-1, 4).sum(axis=-1)
        new_weights = weights.reshape(-1, 4).mean(axis=-1)
    return coarse_field, new_weights

In [5]:
az1, wz1 = hp_coarsen_with_weights(az2)
az0, wz0 = hp_coarsen_with_weights(az1, wz1)

print(np.nanmean(az2))
print(np.average(az1, weights=wz1))
print(np.average(az0, weights=wz0))
print('all_close:', arr_all_close([np.nanmean(az2), np.average(az1, weights=wz1), np.average(az0, weights=wz0)]))

95.96858638743456
95.96858638743456
95.96858638743456
all_close: True


In [6]:
# More realistic, larger data
z = 10
az10 = np.arange(12 * 4**z).astype(float)
# Set ~1% of values to nans.
r = np.random.randint(0, len(az10), 100000)
r.sort()
az10[r] = np.nan

new_field = az10
new_weights = None
glob_means = [np.nanmean(az10)]

print()
print(np.nanmean(az10))
for i in range(11)[::-1]:
    new_field, new_weights = hp_coarsen_with_weights(new_field, new_weights)
    glob_mean = np.sum(new_field * new_weights) / new_weights.sum()
    print(i, glob_mean)
    glob_means.append(glob_mean)
    
print('weighted all_close:', arr_all_close(glob_means))

print()
glob_means = [np.nanmean(az10)]
nf2 = az10  # N.B. az10 unmodified.
print(np.nanmean(az10))
for i in range(11)[::-1]:
    nf2 = np.nanmean(nf2.reshape(-1, 4), axis=-1)
    glob_mean = np.nanmean(nf2)
    print(i, glob_mean)
    glob_means.append(glob_mean)
print(glob_means)
print('non-weighted all_close:', arr_all_close(glob_means))


6291463.156221369
10 6291463.156221369
9 6291463.156221369
8 6291463.156221369
7 6291463.156221369
6 6291463.156221369
5 6291463.156221369
4 6291463.156221369
3 6291463.156221369
2 6291463.156221369
1 6291463.156221369
0 6291463.156221369
weighted all_close: True

6291463.156221369
10 6291455.500023101
9 6291455.5000231005
8 6291455.5000231005
7 6291455.5000231005
6 6291455.5000231005
5 6291455.5000231005
4 6291455.5000231
3 6291455.5000231
2 6291455.5000231
1 6291455.5000231
0 6291455.5000231005
[6291463.156221369, 6291455.500023101, 6291455.5000231005, 6291455.5000231005, 6291455.5000231005, 6291455.5000231005, 6291455.5000231005, 6291455.5000231, 6291455.5000231, 6291455.5000231, 6291455.5000231, 6291455.5000231005]
non-weighted all_close: True


In [7]:
# Simple xarray version. Note, it was tricky/slow to do the ops in xarray so just pull out the np array and use that.
def xr_hp_coarsen_with_weights(da, weights=None):
    dim_name = da.dims[0]
    if weights is not None:
        coarse_field, new_weights = hp_coarsen_with_weights(da.values, weights.values)
    else:
        coarse_field, new_weights = hp_coarsen_with_weights(da.values, None)
    coarse_da = xr.DataArray(coarse_field, dims=dim_name, coords={dim_name: (dim_name, np.arange(len(coarse_field)))})
    da_weights = xr.DataArray(new_weights, dims=dim_name, coords={dim_name: (dim_name, np.arange(len(coarse_field)))})
    return coarse_da, da_weights

In [8]:
daz10 = xr.DataArray(az10, dims=('cell'), coords={'cell': np.arange(len(az10))})

In [9]:
xr_hp_coarsen_with_weights(daz10, None)

(<xarray.DataArray (cell: 3145728)> Size: 25MB
 array([1.50000000e+00, 5.50000000e+00, 9.50000000e+00, ...,
        1.25829015e+07, 1.25829055e+07, 1.25829095e+07])
 Coordinates:
   * cell     (cell) int64 25MB 0 1 2 3 4 ... 3145724 3145725 3145726 3145727,
 <xarray.DataArray (cell: 3145728)> Size: 25MB
 array([1., 1., 1., ..., 1., 1., 1.])
 Coordinates:
   * cell     (cell) int64 25MB 0 1 2 3 4 ... 3145724 3145725 3145726 3145727)

In [10]:
# Ideas cobbled together from https://stackoverflow.com/questions/62592803/xarray-equivalent-of-np-reshape
# However, not quite working, slow and complicated.
da = xr.DataArray(az10, dims=('cell'), coords={'cell': np.arange(len(az10))})

new_cell_flat = np.arange(len(az10) // 4)
dim_to_mean_flat = np.arange(4)
new_cell, dim_to_mean = np.meshgrid(new_cell_flat, dim_to_mean_flat)

da = da.assign_coords({'new_cell': ('cell', new_cell.flatten()), 'dim_to_mean': ('cell', dim_to_mean.flatten())})
da = da.set_index(new_cell_dim_to_mean=('new_cell', 'dim_to_mean')).unstack('new_cell_dim_to_mean')
#da = da.set_index(new_cell_dim_to_mean=('new_cell', 'dim_to_mean'))
# This *should* have new dims of new_cell, dim_to_mean, but it doesn't. Why not??
da

<xarray.DataArray (cell: 12582912)> Size: 101MB
array([0.0000000e+00, 1.0000000e+00, 2.0000000e+00, ..., 1.2582909e+07,
       1.2582910e+07, 1.2582911e+07])
Coordinates:
  * new_cell     (new_cell) int64 25MB 0 1 2 3 ... 3145725 3145726 3145727
  * dim_to_mean  (dim_to_mean) int64 32B 0 1 2 3
  * cell         (cell) int64 101MB 0 1 2 3 ... 12582909 12582910 12582911

In [11]:
da.dims

('cell',)